# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [7]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely

from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [43]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = "/Users/chenruijia/Desktop/AnalyticsTools/NYC-Rental-Analysis/data/drive-download-20231124T164450Z-001/nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "IxFjBjShI6cenJ0NOVZ1rnj0W"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "group14project"
DB_USER = "chenruijia"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [39]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [38]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [37]:
import geopandas as gpd
import pyproj
from functools import partial
from shapely.ops import transform
from shapely.geometry import Polygon

# Assuming the primary .shp file is 'nyc_zipcodes.shp' and it's in the same directory with other related files
shapefile_path = 'data/drive-download-20231124T164450Z-001/nyc_zipcodes.shp'


def load_and_clean_zipcodes(shapefile_path):
    # Reading the shapefile
    geodf_zipcode = gpd.read_file(shapefile_path)

    # Filter out non-Polygon geometries
    geodf_zipcode = geodf_zipcode[geodf_zipcode['geometry'].apply(lambda geom: isinstance(geom, Polygon))]

    # Filter by ZIPCODE pattern (New York City zip codes starting with '1')
    geodf_zipcode = geodf_zipcode[geodf_zipcode['ZIPCODE'].astype(str).str.match(r'^1\d{4}$')]
    
    # Defining the coordinate reference systems
    current_srid = 'EPSG:2263'  # NAD83 / New York Long Island (ftUS)
    desired_srid = 'EPSG:4326'  # WGS 84

    # Updating to use pyproj.Transformer
    transformer = pyproj.Transformer.from_crs(current_srid, desired_srid, always_xy=True)

    # Applying the transformation
    geodf_zipcode['geometry'] = geodf_zipcode['geometry'].apply(lambda geom: transform(transformer.transform, geom))

    return geodf_zipcode


 


In [36]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
def download_and_clean_311_data():
    def fetch_data(offset, limit):
        api_endpoint = f"{base_api_endpoint}?{soql_query}&$select={columns}&$limit={limit}&$offset={offset}"
        response = requests.get(api_endpoint, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            return []
    base_api_endpoint = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
    columns = 'unique_key,created_date,closed_date,complaint_type,incident_zip,latitude,longitude'
    start_date = '2015-01-01T00:00:00'
    end_date = '2023-09-30T23:59:59'
    headers = {'X-App-Token': NYC_DATA_APP_TOKEN}
    soql_query = f"$where=created_date between '{start_date}' and '{end_date}'"
    limit = 200000
    all_data = []
    offset = 0
    more_data = True

    while more_data:
        with ThreadPoolExecutor(max_workers=10) as executor:

            futures = [executor.submit(fetch_data, off, limit) for off in range(offset, offset + 10 * limit, limit)]
    
            for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching Data"):
                data = future.result()
                if data:
                    all_data.extend(data)
                    print(f"Retrieved records up to offset {offset + limit}...")
                else:
                    more_data = False

            offset += 10 * limit
    
    nyc_311_data = pd.DataFrame(all_data)
    nyc_311_data['created_date'] = pd.to_datetime(nyc_311_data['created_date'], format='%Y-%m-%dT%H:%M:%S.%f', errors='coerce')
    nyc_311_data['closed_date'] = pd.to_datetime(nyc_311_data['closed_date'], format='%Y-%m-%dT%H:%M:%S.%f',errors='coerce')
    nyc_311_data = nyc_311_data.dropna(subset=['created_date', 'closed_date'])
    nyc_311_data['longitude'] = nyc_311_data['longitude'].round(3)
    nyc_311_data['latitude'] = nyc_311_data['latitude'].round(3)
    nyc_311_data['incident_zip'] = nyc_311_data['incident_zip'].astype(str)
    nyc_311_data['incident_zip'] = nyc_311_data['incident_zip'].str[:5]
    nyc_311_data = nyc_311_data.dropna(subset=['incident_zip', 'latitude'])
    nyc_311_data.reset_index(drop=True, inplace=True)
    nyc_311_data['id'] = nyc_311_data.index
    columns_to_check_duplicates = ['unique_key']
    nyc_311_data.drop_duplicates(subset=columns_to_check_duplicates, inplace=True)
    
    return nyc_311_data
 

In [35]:
def download_and_clean_tree_data():
    base_api_endpoint = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json' 
    columns = 'tree_id,latitude,longitude,status,health,spc_common,zipcode'
    
    headers = {
        'X-App-Token': NYC_DATA_APP_TOKEN
    }
    
    limit = 50000   
    offset = 0
    all_data = []

    while True:
        api_endpoint = f"{base_api_endpoint}?$select={columns}&$limit={limit}&$offset={offset}"
        response = requests.get(api_endpoint, headers=headers)
        
        if response.status_code == 200:
            page_data = response.json()
    
            if not page_data:
                break
            all_data.extend(page_data)
            offset += limit
            print(f"Retrieved {offset} records so far...")
        else:
            print("Failed to retrieve data:", response.status_code)
            break
    
    geodf_tree_data = pd.DataFrame(all_data)
    geodf_tree_data.dropna(subset=['tree_id', 'latitude', 'longitude', 'status', 'health', 'spc_common', 'zipcode'], inplace=True)
    return geodf_tree_data

In [34]:
def load_and_clean_zillow_data():
    zillow_rent_data = pd.read_csv('data/drive-download-20231124T164450Z-001/zillow_rent_data.csv')

    
    nyc_zip_codes = zillow_rent_data[zillow_rent_data['RegionName'].astype(str).str.match(r'^1\d{4}$')]

    columns_to_delete = ["RegionID", "SizeRank", "RegionType", "StateName", "State", "City", "Metro", "CountyName"]
    df = nyc_zip_codes.drop(columns=columns_to_delete)

    df_long = df.melt(id_vars=["RegionName"], 
                    var_name="Date", 
                    value_name="RentPrice")


    df_long['Date'] = pd.to_datetime(df_long['Date'])

    df_long = df_long.sort_values(by=['RegionName', 'Date'])

    
    df_long['RentPrice'] = df_long.groupby('RegionName')['RentPrice'].ffill()
    df_long['RentPrice'] = df_long.groupby('RegionName')['RentPrice'].bfill()
    df_long['RentPrice'] = df_long.groupby('RegionName')['RentPrice'].ffill().bfill().interpolate(method='linear')

    zillow_data = pd.DataFrame(df_long)
    return zillow_data

 

In [31]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )


In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()


In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

,tree_id,latitude,longitude,status,health,spc_common,zipcode
0,180683,40.72309177,-73.84421522,Alive,Fair,red maple,11375
1,200540,40.79411067,-73.81867946,Alive,Fair,pin oak,11357
2,204026,40.71758074,-73.9366077,Alive,Good,honeylocust,11211
3,204337,40.71353749,-73.93445616,Alive,Good,honeylocust,11211
4,189565,40.66677776,-73.97597938,Alive,Good,American linden,11215


In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

,RegionName,Date,RentPrice
311,10001,2015-01-31,3807.657462
867,10001,2015-02-28,3851.098684
1423,10001,2015-03-31,3844.716691
1979,10001,2015-04-30,3906.689196
2535,10001,2015-05-31,3960.689870


## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

#### Option 2: SQLAlchemy

In [ ]:
!createdb group14

In [ ]:
!psql --dbname group14project -c 'CREATE EXTENSION postgis;'

In [ ]:
!pip install psycopg2
!pip install sqlalchemy pandas geoalchemy2


In [15]:
from geoalchemy2 import Geometry

In [71]:
Base = declarative_base()
engine = create_engine('postgresql://chenruijia:@localhost/group14project')

/var/folders/dn/w_0bwtz955bbs9x6hcsffb540000gn/T/ipykernel_13531/1203867692.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [72]:

# create Geo Zip Code table
class GeoZipCode(Base):
    __tablename__ = 'geo_zip_codes'
    id = Column(Integer, primary_key=True, autoincrement=True, nullable=False)
    ZIPCODE = Column(Integer, primary_key=True)
    BLDGZIP = Column(String)
    PO_NAME = Column(String)
    POPULATION = Column(Float)
    AREA = Column(Float)
    STATE = Column(String)
    COUNTY = Column(String)
    ST_FIPS = Column(Integer)
    CTY_FIPS = Column(Integer)
    URL = Column(String)
    SHAPE_AREA = Column(Float)
    SHAPE_LEN = Column(Float)
    geometry = Column(Geometry(geometry_type='POLYGON'))

# create Tree Census table
class TreeCensus(Base):
    __tablename__ = 'tree_censuses'
    id = Column(Integer, primary_key=True)
    tree_id = Column(Integer, primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    status = Column(String)
    health = Column(String)
    spc_common = Column(String)
    zipcode = Column(Integer)

# create Zillow Rent table
class ZillowRent(Base):
    __tablename__ = 'zillow_rents'
    id = Column(Integer, primary_key=True)
    RegionName = Column(String, primary_key=True)
    Date = Column(Date)
    RentPrice = Column(Float)
    
# create NYC 311 table
class NYC311(Base):
    __tablename__ = 'nyc_311_services'
    id = Column(BigInteger, primary_key=True)
    unique_key = Column(VARCHAR(8), unique=True, primary_key=True)
    created_date = Column(DateTime(timezone=True))
    closed_date = Column(DateTime(timezone=True))
    complaint_type = Column(String)
    incident_zip = Column(VARCHAR(5))
    latitude = Column(Float)
    longitude = Column(Float)

In [78]:
Base.metadata.create_all(engine)

In [75]:
Base.metadata.tables

FacadeDict({'geo_zip_codes': Table('geo_zip_codes', MetaData(), Column('id', Integer(), table=<geo_zip_codes>, primary_key=True, nullable=False), Column('ZIPCODE', Integer(), table=<geo_zip_codes>, primary_key=True, nullable=False), Column('BLDGZIP', String(), table=<geo_zip_codes>), Column('PO_NAME', String(), table=<geo_zip_codes>), Column('POPULATION', Float(), table=<geo_zip_codes>), Column('AREA', Float(), table=<geo_zip_codes>), Column('STATE', String(), table=<geo_zip_codes>), Column('COUNTY', String(), table=<geo_zip_codes>), Column('ST_FIPS', Integer(), table=<geo_zip_codes>), Column('CTY_FIPS', Integer(), table=<geo_zip_codes>), Column('URL', String(), table=<geo_zip_codes>), Column('SHAPE_AREA', Float(), table=<geo_zip_codes>), Column('SHAPE_LEN', Float(), table=<geo_zip_codes>), Column('geometry', Geometry(geometry_type='POLYGON', from_text='ST_GeomFromEWKT', name='geometry'), table=<geo_zip_codes>), schema=None), 'tree_censuses': Table('tree_censuses', MetaData(), Column('

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 2: SQLAlchemy

In [73]:
Session = sessionmaker(bind=engine)
session = Session()

In [80]:
# Insert data into the geo_zip_codes table

from geoalchemy2 import WKTElement
import shapely.wkt

geodf_zipcode = geodf_zipcode[[col for col in geodf_zipcode.columns if col != 'id']]
zipcode_data_list = geodf_zipcode.to_dict(orient='records')

for data in zipcode_data_list:
    geometry_data = data.pop('geometry')

    # Convert Shapely geometry to WKT
    wkt_representation = shapely.wkt.dumps(geometry_data)
    
    # Create a GeoAlchemy WKTElement from the WKT representation
    wkt_element = WKTElement(wkt_representation, srid=4326)
    
    # Add the geometry data back to the dictionary
    data['geometry'] = wkt_element
    
    zipcode = GeoZipCode(**data)
    session.add(zipcode)
session.commit()

In [79]:
session.rollback()

In [ ]:
# Query the geo_zip_codes table

zip_code_rows = session.query(GeoZipCode).all()
print("Number of rows:", len(zip_code_rows))
for result in zip_code_rows:
    print(result.__dict__)

In [ ]:
# Insert data into the tree_censuses table

geodf_tree_data['id'] = range(len(geodf_tree_data))
geodf_tree_data = geodf_tree_data[['id'] + [col for col in geodf_tree_data.columns if col != 'id']]
tree_data_list = geodf_tree_data.to_dict(orient='records')

for data in tree_data_list:
    tree_census = TreeCensus(**data)
    session.add(tree_census)

session.commit()

In [ ]:
# Query the tree_censuses table
tree_census_rows = session.query(TreeCensus).all()
print("Number of rows:", len(tree_census_rows))
for result in tree_census_rows:
    print(result.__dict__)

In [ ]:
# Insert data into the zillow rent table

zillow_rent_data = df_long
zillow_rent_data['id'] = range(len(zillow_rent_data))
zillow_rent_data = zillow_rent_data[['id'] + [col for col in zillow_rent_data.columns if col != 'id']]
zillow_rent_data_list = zillow_rent_data.to_dict(orient='records')

for data in zillow_rent_data_list:
    zillow_rent = ZillowRent(**data)
    session.add(zillow_rent)

session.commit()

In [ ]:
# Query the zillow_rents table

zillow_rent_rows = session.query(ZillowRent).all()
print("Number of rows:", len(zillow_rent_rows))
for result in zillow_rent_rows:
    print(result.__dict__)

In [ ]:
# Insert data into the nyc_311_services table

chunk_size = 10000
# Iterate over the DataFrame in chunks
for start_idx in range(0, len(nyc_311_data), chunk_size):
    end_idx = start_idx + chunk_size
    chunk_data = nyc_311_data.iloc[start_idx:end_idx]

    for index, row in chunk_data.iterrows():
        nyc311 = NYC311(
            id = row['id'],
            unique_key=row['unique_key'],
            created_date=row['created_date'],
            closed_date=row['closed_date'],
            complaint_type=row['complaint_type'],
            incident_zip=row['incident_zip'],
            latitude=row['latitude'],
            longitude=row['longitude']
        )
        session.add(nyc311)

    # Commit changes after processing each chunk
    session.commit()

In [ ]:
# Query the tree_censuses table

tree_census_rows = session.query(TreeCensus).all()
print("Number of rows:", len(tree_census_rows))
for result in tree_census_rows:
    print(result)

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

from sqlalchemy import text
QUERY_1 = text("""
    SELECT incident_zip, COUNT(*) AS num_complaints
    FROM nyc_311_services
    WHERE created_date >= '2022-10-01' AND created_date <= '2023-09-30'
    GROUP BY incident_zip
    ORDER BY num_complaints DESC;
""")

In [ ]:
with engine.connect() as conn:
    result = connection.execute(sql_query)
    print("Zip Code | Number of Complaints")
    print("-" * 30)
    for row in result:
        print(f"{row.incident_zip} | {row.num_complaints}")

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 4

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)